In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import Ridge

In [2]:
df = pd.read_csv("State_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv")
df.drop(['RegionID', 'SizeRank', 'RegionType', 'StateName'], axis=1, inplace=True)
df.head()

,RegionName,1996-01-31,1996-02-29,1996-03-31,1996-04-30,1996-05-31,1996-06-30,1996-07-31,1996-08-31,1996-09-30,...,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30
0,California,160569.0,160342.0,160195.0,159908.0,159714.0,159561.0,159467.0,159452.0,159478.0,...,589875.0,596686.0,603922.0,611570.0,619542.0,628206.0,637594.0,649255.0,665203.0,683996.0
1,Texas,98663.0,98688.0,98717.0,98844.0,98983.0,99100.0,99189.0,99312.0,99467.0,...,219097.0,221039.0,223260.0,225708.0,228092.0,230952.0,234009.0,237786.0,242119.0,247210.0
2,New York,138610.0,138296.0,138111.0,137831.0,137729.0,137663.0,137584.0,137502.0,137465.0,...,336932.0,339991.0,343285.0,346645.0,349903.0,353342.0,357017.0,361111.0,366121.0,371880.0
3,Florida,101472.0,101569.0,101682.0,101894.0,102097.0,102329.0,102551.0,102752.0,102883.0,...,262308.0,264696.0,267233.0,270102.0,273294.0,276745.0,280653.0,285154.0,290727.0,297390.0
4,Illinois,127993.0,127980.0,127893.0,127957.0,127876.0,127995.0,128017.0,128878.0,129969.0,...,212960.0,214764.0,216668.0,218569.0,220297.0,222445.0,224610.0,227345.0,230249.0,233661.0


In [3]:
temp = df.set_index('RegionName')
transposed = temp.T
transposed.index.name = "Date"
transposed.columns.name = None
transposed = transposed.reset_index()
transposed = transposed.reset_index()
transposed['Date'] = pd.to_datetime(transposed['Date'])
transposed = transposed.set_index('Date')
transposed.rename(columns={'index':'month#'}, inplace=True)
transposed

,month#,California,Texas,New York,Florida,Illinois,Pennsylvania,Ohio,Michigan,Georgia,...,New Hampshire,Rhode Island,Montana,Delaware,South Dakota,Alaska,North Dakota,Vermont,District of Columbia,Wyoming
Date,,,,,,,,,,,,,,,,,,,,,
1996-01-31,0,160569.0,98663.0,138610.0,101472.0,127993.0,97541.0,93596.0,91595.0,104500.0,...,107694.0,126769.0,NaN,134517.0,103230.0,108365.0,NaN,120203.0,169270.0,NaN
1996-02-29,1,160342.0,98688.0,138296.0,101569.0,127980.0,97473.0,93709.0,91922.0,104661.0,...,107388.0,126864.0,NaN,134460.0,103544.0,108688.0,NaN,120029.0,168500.0,NaN
1996-03-31,2,160195.0,98717.0,138111.0,101682.0,127893.0,97406.0,93857.0,92234.0,104826.0,...,107180.0,126859.0,NaN,134399.0,103342.0,108887.0,NaN,119782.0,167819.0,NaN
1996-04-30,3,159908.0,98844.0,137831.0,101894.0,127957.0,97295.0,94166.0,92907.0,105182.0,...,106845.0,126862.0,NaN,134254.0,103109.0,109393.0,NaN,119342.0,166654.0,NaN
1996-05-31,4,159714.0,98983.0,137729.0,102097.0,127876.0,97195.0,94487.0,93572.0,105540.0,...,106827.0,126698.0,NaN,134372.0,102580.0,109773.0,NaN,118878.0,165741.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-28,301,628206.0,230952.0,353342.0,276745.0,222445.0,220829.0,172267.0,195598.0,229930.0,...,339322.0,346178.0,331559.0,291007.0,231519.0,295251.0,241035.0,287317.0,669362.0,267947.0
2021-03-31,302,637594.0,234009.0,357017.0,280653.0,224610.0,223639.0,174395.0,197999.0,233025.0,...,344317.0,351708.0,337208.0,294673.0,233987.0,296059.0,242375.0,290222.0,673101.0,269204.0
2021-04-30,303,649255.0,237786.0,361111.0,285154.0,227345.0,226794.0,176723.0,200692.0,236602.0,...,350636.0,357409.0,343855.0,298683.0,236733.0,297542.0,243615.0,293666.0,677323.0,270623.0


In [4]:
predicted_96_04 = transposed.loc[:,['month#']]
ridge_predicted_96_04 = transposed.loc[:,['month#']]

In [5]:
for state in ['California', 'Texas', 'New York', 'Florida', 'Illinois',
       'Pennsylvania', 'Ohio', 'Michigan', 'Georgia', 'North Carolina',
       'New Jersey', 'Virginia', 'Washington', 'Massachusetts', 'Indiana',
       'Arizona', 'Tennessee', 'Missouri', 'Maryland', 'Wisconsin',
       'Minnesota', 'Colorado', 'Alabama', 'South Carolina', 'Louisiana',
       'Kentucky', 'Oregon', 'Oklahoma', 'Connecticut', 'Iowa', 'Mississippi',
       'Arkansas', 'Kansas', 'Utah', 'Nevada', 'West Virginia',
       'Nebraska', 'Idaho', 'Hawaii', 'Maine', 'New Hampshire', 'Rhode Island',
       'Delaware', 'South Dakota', 'Alaska', 'Vermont', 'District of Columbia']:
    state_df = transposed[[state, 'month#']]
    #print(state_df)

    # put month numbers and house values for first 10 years into X and y respectively
    X = state_df.iloc[0:96, 1].values.reshape(-1, 1)
    y = state_df.iloc[0:96, 0].values.reshape(-1, 1)

    X_future = state_df.iloc[120:307, 1].values.reshape(-1, 1)
    y_future = state_df.iloc[120:307, 0].values.reshape(-1, 1)

    # Create the model
    model = LinearRegression()
    ridge_model = Ridge(alpha=100000.0)#100,000

    # Fit the model to the training data. 
    model.fit(X, y)
    ridge_model.fit(X, y)

    predicted_96_04[state] = model.predict(state_df['month#'].values.reshape(-1, 1))
    ridge_predicted_96_04[state + 'ridge'] = ridge_model.predict(state_df['month#'].values.reshape(-1, 1))
    
predicted_96_04.drop(['month#'], axis=1, inplace=True)
ridge_predicted_96_04.drop(['month#'], axis=1, inplace=True)
predicted_96_04 = predicted_96_04.T
ridge_predicted_96_04 = ridge_predicted_96_04.T
predicted_96_04.index.name = 'RegionName'
ridge_predicted_96_04.index.name = 'RegionName'

predicted_96_04.to_csv('home_value_files/9604_State_Home_Values_LINEARpredicted.csv')
#ridge_predicted_96_04.to_csv('9604_State_Home_Values_RIDGEpredicted.csv')
predicted_96_04

Date,1996-01-31,1996-02-29,1996-03-31,1996-04-30,1996-05-31,1996-06-30,1996-07-31,1996-08-31,1996-09-30,1996-10-31,...,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30
RegionName,,,,,,,,,,,,,,,,,,,,,
California,127490.201890,129400.665184,131311.128477,133221.591771,135132.055064,137042.518358,138952.981651,140863.444945,142773.908238,144684.371532,...,692987.336779,694897.800072,696808.263366,698718.726659,700629.189953,702539.653247,704450.116540,706360.579834,708271.043127,710181.506421
Texas,97021.833978,97283.810938,97545.787898,97807.764858,98069.741818,98331.718778,98593.695738,98855.672698,99117.649659,99379.626619,...,174567.014173,174828.991133,175090.968093,175352.945053,175614.922013,175876.898974,176138.875934,176400.852894,176662.829854,176924.806814
New York,122142.784364,123032.801185,123922.818005,124812.834825,125702.851646,126592.868466,127482.885287,128372.902107,129262.918927,130152.935748,...,385587.763203,386477.780024,387367.796844,388257.813664,389147.830485,390037.847305,390927.864126,391817.880946,392707.897766,393597.914587
Florida,91280.105026,91936.372552,92592.640077,93248.907603,93905.175129,94561.442655,95217.710180,95873.977706,96530.245232,97186.512758,...,285535.292655,286191.560180,286847.827706,287504.095232,288160.362758,288816.630284,289472.897809,290129.165335,290785.432861,291441.700387
Illinois,115948.903351,116702.625780,117456.348209,118210.070639,118963.793068,119717.515498,120471.237927,121224.960357,121978.682786,122732.405216,...,339050.742472,339804.464901,340558.187330,341311.909760,342065.632189,342819.354619,343573.077048,344326.799478,345080.521907,345834.244337
Pennsylvania,90953.154854,91337.932296,91722.709737,92107.487179,92492.264621,92877.042062,93261.819504,93646.596946,94031.374387,94416.151829,...,204847.277589,205232.055030,205616.832472,206001.609914,206386.387355,206771.164797,207155.942239,207540.719680,207925.497122,208310.274564
Ohio,92313.157002,92679.032863,93044.908724,93410.784586,93776.660447,94142.536309,94508.412170,94874.288031,95240.163893,95606.039754,...,200612.411966,200978.287828,201344.163689,201710.039551,202075.915412,202441.791273,202807.667135,203173.542996,203539.418857,203905.294719
Michigan,91082.658720,91721.752528,92360.846335,92999.940143,93639.033951,94278.127758,94917.221566,95556.315373,96195.409181,96834.502989,...,280254.425784,280893.519592,281532.613400,282171.707207,282810.801015,283449.894823,284088.988630,284728.082438,285367.176246,286006.270053
Georgia,101016.227663,101637.396335,102258.565007,102879.733679,103500.902351,104122.071023,104743.239695,105364.408367,105985.577039,106606.745711,...,284882.154576,285503.323248,286124.491920,286745.660592,287366.829264,287987.997936,288609.166608,289230.335280,289851.503952,290472.672624


In [6]:
predicted_11_19 = transposed.loc[:,['month#']]
ridge_predicted_11_19 = transposed.loc[:,['month#']]

for state in ['California', 'Texas', 'New York', 'Florida', 'Illinois',
       'Pennsylvania', 'Ohio', 'Michigan', 'Georgia', 'North Carolina',
       'New Jersey', 'Virginia', 'Washington', 'Massachusetts', 'Indiana',
       'Arizona', 'Tennessee', 'Missouri', 'Maryland', 'Wisconsin',
       'Minnesota', 'Colorado', 'Alabama', 'South Carolina', 'Louisiana',
       'Kentucky', 'Oregon', 'Oklahoma', 'Connecticut', 'Iowa', 'Mississippi',
       'Arkansas', 'Kansas', 'Utah', 'Nevada', 'New Mexico', 'West Virginia',
       'Nebraska', 'Idaho', 'Hawaii', 'Maine', 'New Hampshire', 'Rhode Island',
       'Montana', 'Delaware', 'South Dakota', 'Alaska', 'North Dakota',
       'Vermont', 'District of Columbia', 'Wyoming']:
    state_df = transposed[[state, 'month#']]
    #print(state_df)

    # put month numbers and house values for 2011-2019 into X and y respectively
    X = state_df.iloc[170:276, 1].values.reshape(-1, 1)
    y = state_df.iloc[170:276, 0].values.reshape(-1, 1)

    X_future = state_df.iloc[120:307, 1].values.reshape(-1, 1)
    y_future = state_df.iloc[120:307, 0].values.reshape(-1, 1)

    # Create the model
    model = LinearRegression()
    ridge_model = Ridge(alpha=100000.0)#100,000

    # Fit the model to the training data. 
    model.fit(X, y)
    ridge_model.fit(X, y)

    predicted_11_19[state] = model.predict(state_df['month#'].values.reshape(-1, 1))
    ridge_predicted_11_19[state + 'ridge'] = ridge_model.predict(state_df['month#'].values.reshape(-1, 1))

predicted_11_19 = predicted_11_19[predicted_11_19['month#'] > 167]
ridge_predicted_11_19 = ridge_predicted_11_19[ridge_predicted_11_19['month#'] > 167]
predicted_11_19.drop(['month#'], axis=1, inplace=True)
ridge_predicted_11_19.drop(['month#'], axis=1, inplace=True)
predicted_11_19 = predicted_11_19.T
ridge_predicted_11_19 = ridge_predicted_11_19.T
predicted_11_19.index.name = 'RegionName'
ridge_predicted_11_19.index.name = 'RegionName'

predicted_11_19.to_csv('home_value_files/1119_State_Home_Values_LINEARpredicted.csv')
#ridge_predicted_11_19.to_csv('1119_State_Home_Values_RIDGEpredicted.csv')
predicted_11_19

Date,2010-01-31,2010-02-28,2010-03-31,2010-04-30,2010-05-31,2010-06-30,2010-07-31,2010-08-31,2010-09-30,2010-10-31,...,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30
RegionName,,,,,,,,,,,,,,,,,,,,,
California,270679.821322,273283.517520,275887.213719,278490.909918,281094.606116,283698.302315,286301.998514,288905.694712,291509.390911,294113.087110,...,603952.934756,606556.630954,609160.327153,611764.023352,614367.719551,616971.415749,619575.111948,622178.808147,624782.504345,627386.200544
Texas,119489.242930,120223.054987,120956.867043,121690.679099,122424.491156,123158.303212,123892.115268,124625.927324,125359.739381,126093.551437,...,213417.186140,214150.998196,214884.810253,215618.622309,216352.434365,217086.246422,217820.058478,218553.870534,219287.682591,220021.494647
New York,244422.546082,244938.914286,245455.282490,245971.650694,246488.018898,247004.387102,247520.755306,248037.123511,248553.491715,249069.859919,...,310517.676212,311034.044416,311550.412621,312066.780825,312583.149029,313099.517233,313615.885437,314132.253641,314648.621845,315164.990050
Florida,125157.052578,126191.425337,127225.798096,128260.170854,129294.543613,130328.916372,131363.289130,132397.661889,133432.034647,134466.407406,...,257556.765685,258591.138444,259625.511202,260659.883961,261694.256720,262728.629478,263763.002237,264797.374996,265831.747754,266866.120513
Illinois,159421.519682,159766.849596,160112.179510,160457.509424,160802.839338,161148.169252,161493.499166,161838.829080,162184.158994,162529.488908,...,203623.748686,203969.078600,204314.408514,204659.738429,205005.068343,205350.398257,205695.728171,206041.058085,206386.387999,206731.717913
Pennsylvania,157527.108280,157736.141779,157945.175278,158154.208776,158363.242275,158572.275774,158781.309273,158990.342771,159199.376270,159408.409769,...,184283.396121,184492.429619,184701.463118,184910.496617,185119.530116,185328.563614,185537.597113,185746.630612,185955.664111,186164.697609
Ohio,107487.587158,107749.168733,108010.750309,108272.331884,108533.913459,108795.495035,109057.076610,109318.658186,109580.239761,109841.821337,...,140970.028813,141231.610389,141493.191964,141754.773540,142016.355115,142277.936690,142539.518266,142801.099841,143062.681417,143324.262992
Michigan,86339.894405,87036.066846,87732.239288,88428.411729,89124.584170,89820.756611,90516.929053,91213.101494,91909.273935,92605.446376,...,175449.966884,176146.139325,176842.311767,177538.484208,178234.656649,178930.829090,179627.001532,180323.173973,181019.346414,181715.518855
Georgia,115815.722513,116464.214016,117112.705519,117761.197022,118409.688526,119058.180029,119706.671532,120355.163035,121003.654538,121652.146041,...,198822.634914,199471.126418,200119.617921,200768.109424,201416.600927,202065.092430,202713.583933,203362.075436,204010.566940,204659.058443


In [9]:
table_df = df.copy()
table_df.set_index('RegionName', inplace=True)
table_df.index.name = 'State'
table_df.drop(df.columns[1:306], axis=1, inplace=True)
table_df['11-19 Growth'] = predicted_11_19['2021-06-30']
table_df['96-04 Growth'] = predicted_96_04['2021-06-30']
table_df.rename(columns={'2021-06-30':'Actual'}, inplace=True)
table_df["11-19 Difference"] = table_df['Actual'] - table_df['11-19 Growth']
table_df["96-04 Difference"] = table_df['Actual'] - table_df['96-04 Growth']
table_df.to_csv('home_value_files/All_June_2021_Values.csv')
table_df

,Actual,11-19 Growth,96-04 Growth,11-19 Difference,96-04 Difference
State,,,,,
California,683996.0,627386.200544,710181.506421,56609.799456,-26185.506421
Texas,247210.0,220021.494647,176924.806814,27188.505353,70285.193186
New York,371880.0,315164.990050,393597.914587,56715.009950,-21717.914587
Florida,297390.0,266866.120513,291441.700387,30523.879487,5948.299613
Illinois,233661.0,206731.717913,345834.244337,26929.282087,-112173.244337
Pennsylvania,234684.0,186164.697609,208310.274564,48519.302391,26373.725436
Ohio,181756.0,143324.262992,203905.294719,38431.737008,-22149.294719
Michigan,208337.0,181715.518855,286006.270053,26621.481145,-77669.270053
Georgia,245778.0,204659.058443,290472.672624,41118.941557,-44694.672624
